In [ ]:
import numpy as np
import pandas as pd
import xlwings as xw
import re

In [ ]:
# DRM이 적용된 파일을 읽기 위해 xlwings 사용
# xlwings 인스턴스 생성
app = xw.App(visible=False)

# CSV 파일 로드
wb1 = app.books.open('./네이버 MX_통합.csv')
sheet1 = wb1.sheets[0].used_range.value
df_mx_all = pd.DataFrame(sheet1)
df_mx_all.columns = df_mx_all.iloc[0,:]
df_mx_all = df_mx_all.iloc[1:,:]

wb2 = app.books.open('./갤럭시폴드5.csv')
sheet2 = wb2.sheets[0].used_range.value
df_fold5 = pd.DataFrame(sheet2)
df_fold5.columns = df_fold5.iloc[0,:]
df_fold5 = df_fold5.iloc[1:,:]

wb3 = app.books.open('./갤럭시플립5.csv')
sheet3 = wb3.sheets[0].used_range.value
df_flip5 = pd.DataFrame(sheet3)
df_flip5.columns = df_flip5.iloc[0,:]
df_flip5 = df_flip5.iloc[1:,:]

# xlwings 인스턴스 종료
app.kill()

In [ ]:
# '휴대폰' 카테고리 데이터 필터링
df_mx_smartphone = df_mx_all[df_mx_all['카테고리'].str.contains('디지털/가전>휴대폰>', na=False)]

In [ ]:
# 폴드5, 플립5 데이터프레임의 컬럼을 통일
df_fold5 = df_fold5[df_mx_smartphone.columns]
df_flip5 = df_flip5[df_mx_smartphone.columns]

In [ ]:
# 모든 데이터프레임 병합
df_reviews = pd.concat([df_mx_smartphone, df_fold5, df_flip5])

# '해외' 키워드가 포함된 제품 제외
df_reviews = df_reviews[~df_reviews['제품명'].str.contains('해외', na=False)]

In [ ]:
# 리뷰 수가 20개 이상인 제품만 필터링
product_count = df_reviews['제품명'].value_counts().reset_index()
product_count_over_20 = product_count[product_count['count']>20]
product_count_over_20_list = product_count_over_20['제품명'].to_list()

df_reviews = df_reviews[df_reviews['제품명'].isin(product_count_over_20_list)]

In [ ]:
# 날짜 데이터 형식 변환
df_reviews['날짜'] = pd.to_datetime(df_reviews['날짜'], errors='coerce')

# 제품명 정제: 불필요한 키워드 제거
df_reviews['제품명'] = (df_reviews['제품명']
                      .str.replace('[자급제]', '', regex=False)
                      .str.replace('(자급제)', '', regex=False)
                      .str.replace('[KT]', '', regex=False)
                      .str.replace('[SKT]', '', regex=False)
                      .str.replace('(LG U+)', '', regex=False)
                      .str.replace('(SKT)', '', regex=False)
                      .str.replace('(KT)', '', regex=False)
                      .str.replace('삼성전자', '', regex=False)
                      .str.replace('Apple', '', regex=False)
                      .str.strip())


In [ ]:
# 브랜드명 정제
df_reviews['브랜드'] = (df_reviews['브랜드']
                    .str.replace('제조사', '', regex=False)
                    .str.replace('브랜드', '', regex=False)
                    .str.replace('갤럭시', '삼성전자', regex=False)
                    .str.replace('Apple', '애플', regex=False)
                    .str.strip())


In [ ]:
# 리뷰 길이를 기준으로 제품별 순위 부여 (상위 50개 필터링)
df_reviews['review_len'] = df_reviews['리뷰'].str.len()
df_reviews['len_rank'] = df_reviews.groupby('제품명')['review_len'].rank(method='first', ascending=False)
df_reviews = df_reviews[df_reviews['len_rank'] <= 50]

In [ ]:
# 인덱스 재설정 및 리뷰 번호 부여
df_reviews.reset_index(drop=True, inplace=True)
df_reviews.reset_index(inplace=True)
df_reviews.rename(columns={'index':'review_number'}, inplace=True)

In [ ]:
# 정제된 리뷰 데이터프레임 저장
df_reviews.to_pickle('./reviews.pickle')

--- 
## 모델 정보 데이터셋 생성

In [ ]:
# 모델 정보 추출을 위한 데이터프레임 복사 및 중복 제거
df_models = df_reviews[['브랜드','제품명']].copy()
df_models.drop_duplicates(inplace=True)
df_models.reset_index(drop=True, inplace=True)

In [ ]:
# 정규표현식을 사용하여 용량 정보 추출
pattern = r'(\d+(?:GB|TB))'
df_models['용량'] = df_models['제품명'].str.extract(pattern)

In [ ]:
# 제품명에서 용량 정보를 제거하여 기본 모델명 추출
storage_list = df_models['용량'].unique().tolist()
df_models['모델'] = df_models['제품명']

for storage_name in storage_list:
    if pd.notna(storage_name):
        df_models['모델'] = df_models['모델'].str.replace(storage_name, '', regex=False)

In [ ]:
# 모델명 정제: 부가 정보(5G, 플러스 등) 제거
df_models['모델'] = (df_models['모델']
                   .str.replace('5G', '', regex=False)
                   .str.replace('플러스', '', regex=False)
                   .str.replace('울트라', '', regex=False)
                   .str.replace('프로', '', regex=False)
                   .str.replace('미니', '', regex=False)
                   .str.replace('맥스', '', regex=False)
                   .str.replace('FE', '', regex=False)
                   .str.replace('A52S', 'A52', regex=False) # A52S -> A52 통일
                   .str.strip())


In [ ]:
# 제품명(용량x) 컬럼 생성
df_models['제품명(용량x)'] = df_models['제품명']
for storage_name in storage_list:
    if pd.notna(storage_name):
        df_models['제품명(용량x)'] = df_models['제품명(용량x)'].str.replace(storage_name, '', regex=False)

In [ ]:
# 모델명을 기준으로 시리즈 분류
df_models['시리즈'] = np.nan
df_models.loc[df_models['모델'].str.contains('갤럭시A', na=False), '시리즈'] = 'A 시리즈'
df_models.loc[df_models['모델'].str.contains('갤럭시S', na=False), '시리즈'] = 'S 시리즈'
df_models.loc[df_models['모델'].str.contains('노트', na=False), '시리즈'] = '노트 시리즈'
df_models.loc[df_models['모델'].str.contains('폴드|플립', na=False, regex=True), '시리즈'] = '폴더블'
df_models.loc[df_models['모델'].str.contains('아이폰', na=False), '시리즈'] = '아이폰'

In [ ]:
# 최종 컬럼 선택 및 정렬
final_cols = ['제품명', '브랜드', '시리즈', '모델', '용량', '제품명(용량x)']
df_models = df_models[final_cols]
df_models.sort_values(by=['브랜드', '시리즈', '모델', '용량'], inplace=True)
df_models.reset_index(drop=True, inplace=True)

In [ ]:
# 정제된 모델 데이터프레임 저장
df_models.to_pickle('./models.pickle')